In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext autotime

time: 274 µs


In [3]:
%cd ..

/Users/rubenbroekx/Documents/Projects/twitter-sentiment-classifier/twitter_sentiment_classifier
time: 1.18 ms


In [4]:
# Necessary to be in root to get access to store
%cd ..

/Users/rubenbroekx/Documents/Projects/twitter-sentiment-classifier
time: 1.1 ms


# Train Classifier

This notebook will head (classifier) part of the model.

In [5]:
import json
import os
import pandas as pd

from twitter_sentiment_classifier.store.loader import fetch_all_tweet_data
from tqdm import tqdm
from collections import Counter

time: 4.75 s


## 1. Load in data

Load in the different datasets (`train`, `test`, and `validation`).

In [6]:
# Fetch all tweet data from S3 (skips automatically if this is already the case)
fetch_all_tweet_data()

time: 773 µs


In [7]:
# Training
with open(os.path.expanduser('twitter_sentiment_classifier/store/data/tweets_train.jsonl'), 'r') as f:
    annotations_train = [json.loads(line) for line in f.readlines()]
print(f"Loaded in {len(annotations_train)} training annotations")

# Testing
with open(os.path.expanduser('twitter_sentiment_classifier/store/data/tweets_test.jsonl'), 'r') as f:
    annotations_test = [json.loads(line) for line in f.readlines()]
print(f"Loaded in {len(annotations_test)} testing annotations")

# Validation
with open(os.path.expanduser('twitter_sentiment_classifier/store/data/tweets_val.jsonl'), 'r') as f:
    annotations_val = [json.loads(line) for line in f.readlines()]
print(f"Loaded in {len(annotations_val)} validation annotations")

Loaded in 38723 training annotations
Loaded in 3000 testing annotations
Loaded in 1198 validation annotations
time: 796 ms


## 2. Load model

Load in the model used to train. We choose to train the RobBERT model from scratch, to not introduce any biases in a previously trained model.

In [9]:
from twitter_sentiment_classifier.sentiment_model import Classifier

time: 566 µs


In [10]:
# Load in the model
classifier = Classifier()
classifier

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


Classifier(
	n_classes=3
	batch_size=1024
	use_cuda=False
)

time: 4.71 s


## 3. Training

Train part of the model for a limited amount of epochs.

In [10]:
# Number of training epochs
N_EPOCHS = 10

time: 592 µs


In [12]:
classifier.train(
    n_epochs=N_EPOCHS,
    train_samples=annotations_train,
    val_samples=annotations_val,
    test_samples=annotations_test,
)

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.728051,0.758259,0.667780,0.666797,0.667728,0.667780
200,0.723047,0.758172,0.671119,0.669861,0.671332,0.671119
300,0.721877,0.757971,0.666945,0.665896,0.667127,0.666945


PredictionOutput(predictions=array([[-0.9718493 ,  2.5009346 , -1.0385834 ],
       [ 0.14673176,  1.0660126 , -1.3656752 ],
       [-1.5171548 ,  2.1559024 , -0.23238656],
       ...,
       [-1.8455203 , -0.6517119 ,  2.9414067 ],
       [-0.9130719 ,  1.0140048 , -0.05105001],
       [-1.4499953 , -0.9083092 ,  2.7736511 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 2, 1, 2]), metrics={'eval_loss': 0.6688432097434998, 'eval_accuracy': 0.7146666666666667, 'eval_f1': 0.7146277973933322, 'eval_precision': 0.7155947454238136, 'eval_recall': 0.7146666666666667})

time: 55.8 s


## 4. Evaluation

Evaluate this intermediate version of the model. It should already have a decent performance.

In [11]:
classifier.eval(annotations_test)

PredictionOutput(predictions=array([[-1.1905937 ,  2.5736046 , -0.8790978 ],
       [ 0.18675911,  1.0651293 , -1.4068074 ],
       [-1.7247698 ,  2.2494407 , -0.10740978],
       ...,
       [-1.9217447 , -0.62404865,  3.002963  ],
       [-0.96668375,  1.0870813 , -0.05590736],
       [-1.5493615 , -0.87750244,  2.8522382 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 2, 1, 2]), metrics={'eval_loss': 0.6695119142532349, 'eval_accuracy': 0.711, 'eval_f1': 0.7110215714963962, 'eval_precision': 0.7111935119957908, 'eval_recall': 0.711})

time: 3.08 s
